In [ ]:
!git clone https://github.com/Mayraju/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 35 (delta 14), reused 35 (delta 14), pack-reused 0
Unpacking objects: 100% (35/35), done.
Checking out files: 100% (24/24), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense
from keras import optimizers
from keras import backend as K
import json
from sklearn.utils import shuffle
import os
import sys
import time

In [ ]:
#Data class processing
class data_cls:
  def __init__(self,train_test,**kwargs):
     col_names = ["duration","protocol_type","service","flag","src_bytes",
            "dst_bytes","land_f","wrong_fragment","urgent","hot","num_failed_logins",
            "logged_in","num_compromised","root_shell","su_attempted","num_root",
            "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
            "is_host_login","is_guest_login","count","srv_count","serror_rate",
            "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
            "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
            "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
            "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
            "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels","dificulty"]
     self.index = 0
     self.loaded = False
     self.train_test = train_test
     self.train_path = kwargs.get('train_path','/content/datasets/datasets/NSL/KDDTrain+.txt')
     self.test_path = kwargs.get('test_path','/content/datasets/datasets/NSL/KDDTest+.txt')
     self.formated_train_path = kwargs.get('formated_train_path',"/content/datasets/datasets/formated/formated_train_adv.data.txt")
     self.formated_test_path = kwargs.get('formated_test_path',"/content/datasets/datasets/formated/formated_test_adv.data.txt")
     self.attack_types = ['normal','Dos','Probe','R2L','U2R']
     self.attack_names = []
     self.attack_map = {'normal':'normal',
                        
                        'back' :'Dos',
                        'land' :'Dos',
                        'neptune':'Dos',
                        'pod':'Dos',
                        'smurf':'Dos',
                        'teardrop':'Dos',
                        'mailbomb':'Dos',
                        'apache2':'Dos',
                        'processtable':'Dos',
                         'udpstorm':'Dos',
                        
                        'ipsweep':'Probe',
                        'nmap':'Probe',
                        'portsweep':'Probe',
                        'satan':'Probe',
                        'mscan':'Probe',
                        'saint':'Probe',
                        
                        'ftp_write':'R2L',
                        'guess_passwd':'R2L',
                        'imap':'R2L',
                        'multihop':'R2L',
                        'phf':'R2L',
                        'spy':'R2L',
                        'warezclient':'R2L',
                        'warezmaster':'R2L',
                        'sendmail':'R2L',
                        'named':'R2L',
                        'snmpgetattack':'R2L',
                        'snmpguess':'R2L',
                        'xlock':'R2L',
                        'xsnoop':'R2L',
                        'worm':'R2L',
                        
                        
                        'buffer_overflow':'U2R',
                        'loadmodule':'U2R',
                        'perl':'U2R',
                        'rootkit':'U2R',
                        'httptunnel':'U2R',
                        'ps':'U2R',
                        'sqlattack':'U2R',
                        'xterm':'U2R'
                      }
     self.all_attack_names = list(self.attack_map.keys())
     formated = False
     if os.path.exists(self.formated_train_path) and os.path.exists(self.formated_test_path):
       formated = True
     self.formated_dir = "/content/datasets/datasets/formated"
     if not os.path.exists(self.formated_dir):
       os.makedirs(self.formated_dir)
    

    # if it does not exist,it's needed to format the data

     if not formated:
        self.df = pd.read_csv(self.train_path,sep=',',names=col_names,index_col=False) #reading the train data
        if 'dificulty' in self.df.columns:
         self.df.drop('dificulty',axis=1,inplace=True)
        data2 = pd.read_csv(self.test_path,sep=',',names=col_names,index_col=False) #reading the test data
        if 'dificulty' in data2:
           del(data2['dificulty'])
        train_indx = self.df.shape[0]
        frames = [self.df,data2]
        self.df=pd.concat(frames)

        #dataFrame processing
        self.df = pd.concat([self.df.drop('protocol_type',axis=1),pd.get_dummies(self.df['protocol_type'])],axis=1)
        self.df = pd.concat([self.df.drop('service',axis=1),pd.get_dummies(self.df['service'])],axis=1)
        self.df = pd.concat([self.df.drop('flag',axis=1),pd.get_dummies(self.df['flag'])],axis=1)

      # 1 if 'su root' command attempted; 0 otherwise
        self.df['su_attempted'] = self.df['su_attempted'].replace(2.0,0.0)

      #one hot encoding for labels
        self.df = pd.concat([self.df.drop('labels',axis=1),pd.get_dummies(self.df['labels'])],axis=1)

      #Normalization of the df

      #normalizd_df =(df-df.mean())/df.std()

        for indx,dtype in self.df.dtypes.iteritems():
          if dtype == 'float64' or dtype == 'int64':
            if self.df[indx].max() == 0 and self.df[indx].min() == 0:
              self.df[indx] = 0
            else:
              self.df[indx] = (self.df[indx]-self.df[indx].min())/(self.df[indx].max()-self.df[indx].min())
      

      #save data
        test_df = self.df.iloc[train_indx:self.df.shape[0]]
        test_df =shuffle(test_df,random_state=np.random.randint(0,100))
        self.df = self.df[:train_indx]
        self.df = shuffle(self.df,random_state=np.random.randint(0,100))
        test_df.to_csv(self.formated_test_path,sep=',',index=False)
        self.df.to_csv(self.formated_train_path,sep=',',index=False)

        for att in self.attack_map:
          if att in self.df.columns:
            if np.sum(self.df[att].values) > 1:
               self.attack_names.append(att)


  def get_shape(self):
    if self.loaded is False:
      self._load_df()
    self.data_shape = self.df.shape
    
    return self.data_shape

  def get_batch(self,batch_size=100):
        if self.loaded is False:
            self._load_df()
        
        # Read the df rows
        indexes = list(range(self.index,self.index+batch_size))    
        if max(indexes)>self.data_shape[0]-1:
            dif = max(indexes)-self.data_shape[0]
            indexes[len(indexes)-dif-1:len(indexes)] = list(range(dif+1))
            self.index=batch_size-dif
            batch = self.df.iloc[indexes]
        else: 
            batch = self.df.iloc[indexes]
            self.index += batch_size    
            
        labels = batch[self.attack_names]
        print("labels")
        print(self.all_attack_names)
        batch = batch.drop(self.all_attack_names,axis=1)
        print("batch")
        return batch,labels

  def get_full(self):
        if self.loaded is False:
           self._load_df()

        labels = self.df[self.attack_names]
        batch = self.df.drop(self.all_attack_names,axis=1)
        return batch,labels

   # one random sample ni generate chestunam and ani attack types > 1 ni store chestunam
  def _load_df(self):                                    
       if self.train_test == 'train': 
            self.df = pd.read_csv(self.formated_train_path,sep=',')
       else:
            self.df = pd.read_csv(self.formated_test_path,sep=',')
       self.index = np.random.randint(0,self.df.shape[0]-1,dtype=np.int32)  #getting the random state 
       self.loaded = True
       for att in self.attack_map:
         if att in self.df.columns:
            if np.sum(self.df[att].values) > 1:
               self.attack_names.append(att)





In [ ]:
def huber_loss(y_true,y_pred,clip_value=1):

   assert clip_value > 0

   x = y_true - y_pred
   if np.isinf(clip_value):

       return .5 * K.square(x)

    
   condition = K.abs(x) < clip_value
   squared_loss = .5 * K.square(x)
   linear_loss = clip_value * (K.abs(x) - .5 * clip_value)
   if K.backend() == 'tensorflow':
     import tensorflow as tf
     if hasattr(tf,'select'):
       return tf.select(condition,squared_loss,linear_loss)
     else:
       return tf.where(condition,squared_loss,linear_loss)
   elif K.backend() == 'theano':
     from theano import tensor as T
     return T.switch(condition,squared_loss,linear_loss)
   else:
     raise RuntimeError('Unknown backend "{}".'.format(k.backend()))

In [ ]:
import keras.losses
keras.losses.huber_loss = huber_loss

In [ ]:
class QNetwork():

  def __init__(self,obs_size,num_actions,hidden_size=100,hidden_layers = 1,learning_rate = .2):

     self.obs_size = obs_size
     self.num_actions = num_actions
     self.model = Sequential()
     # add input layer
     self.model.add(Dense(hidden_size,input_shape=(obs_size, ),activation='relu')) 
     #add hidden layers
     for layers in range(hidden_layers):
       self.model.add(Dense(hidden_size,activation='relu'))
      #add output layers.
     self.model.add(Dense(num_actions))

     optimizer = optimizers.Adam(0.00025)

     self.model.compile(loss=huber_loss,optimizer = optimizer)

     # predict action values
  def predict(self,state,batch_size=1):

      return self.model.predict(state,batch_size=batch_size)

     #states: Target states
      
      #    q: Estimated values
      #  Returns:
       #   The calculated loss on the batch.
  def update(self,states,q):

    loss = self.model.train_on_batch(states,q)
    return loss


    
  def copy_model(model):

     model.save('tmp_model')
     return keras.models.load_model('tmp_model')

In [ ]:
class Policy:
  def __init__(self,num_actions,estimator):
    self.num_actions = num_actions
    self.estimator = estimator

In [ ]:
class Epsilon_greedy(Policy):

  def __init__(self,estimator,num_actions,epsilon,min_epsilon,decay_rate,epoch_length):
     Policy.__init__(self,num_actions,estimator)
     self.name = "Epsilon Greedy"

     if (epsilon is None or epsilon < 0 or epsilon > 1):
        print("EpsilonGreedy : Invalid value of position",flush = True)
        sys.exit(0)
     self.epsilon = epsilon

     self.min_epsilon = min_epsilon

     self.actions = list(range(num_actions))

     self.step_counter = 0

     self.epoch_length = epoch_length

     self.decay_rate = decay_rate

     if self.epsilon > 0.01:
       self.epsilon_decay = True
     else:
       self.epsilon_decay = False

  def get_actions(self,states):

    if np.random.rand() <= self.epsilon:

       actions = np.random.randint(0,self.num_actions,states.shape[0])
    else:
      self.Q = self.estimator.predict(states,states.shape[0])
      actions = []
      for row in range(self.Q.shape[0]):
         best_actions = np.argwhere(self.Q[row] == np.amax(self.Q[row]))
         actions.append(best_actions[np.random.choice(len(best_actions))].item())
   
    self.step_counter +=1

    if self.epsilon_decay:
      if self.step_counter % self.epoch_length == 0:
         self.epsilon = max(self.min_epsilon,self.epsilon * self.decay_rate**self.step_counter)

    return actions

In [ ]:
class ReplayMemory(object):

  def __init__(self,observation_size,max_size):

    self.observation_size = observation_size
    self.num_observed = 0
    self.max_size = max_size
    self.samples = {
        
         'obs' : np.zeros(self.max_size * 1 * self.observation_size,dtype=np.float32).reshape(self.max_size,self.observation_size),

         'action' : np.zeros(self.max_size * 1 ,dtype=np.int16).reshape(self.max_size,1),
         
         'reward' : np.zeros(self.max_size * 1).reshape(self.max_size,1),

         'terminal' : np.zeros(self.max_size *1 ,dtype=np.int16).reshape(self.max_size,1)
    }
    
  def observe(self,state,action,reward,done):
      index = self.num_observed % self.max_size
      self.samples['obs'][index,:] = state
      self.samples['action'][index,:]=action
      self.samples['reward'][index,:]=reward
      self.samples['terminal'][index,:]=done

      self.num_observed +=1

  def sample_minibatch(self,minibatch_size):
      max_index = min(self.num_observed,self.max_size) - 1
      sampled_indices = np.random.randint(max_index,size = minibatch_size)

      s = np.asarray(self.samples['obs'][sampled_indices,:],dtype = np.float32)
      s_next = np.asarray(self.samples['obs'][sampled_indices+1,:],dtype=np.float32)

      a = self.samples['action'][sampled_indices].reshape(minibatch_size)

      r = self.samples['reward'][sampled_indices].reshape((minibatch_size,1))

      done = self.samples['terminal'][sampled_indices].reshape((minibatch_size,1))

      return (s,a,r,s_next,done)

In [ ]:
class Agent(object):

  def __init__(self,actions,obs_size,policy="EpsilonGreedy",**kwargs):
    self.actions = actions
    self.num_actions = len(actions)
    self.obs_size = obs_size

    self.epsilon = kwargs.get('epsilon',1)
    self.min_epsilon = kwargs.get('min_epsilon',.1)
    self.gamma = kwargs.get('gamma',.001)
    self.minibatch_size = kwargs.get('minibatch_size',2)
    self.epoch_length = kwargs.get('epoch_length',100)
    self.decay_rate = kwargs.get('decay_rate',0.99)
    self.ExpRep = kwargs.get('ExpRep',True)

    if self.ExpRep:

       self.memory = ReplayMemory(self.obs_size,kwargs.get('mem_size',10))
    
    self.ddqn_time = 100
    self.ddqn_update = self.ddqn_time

    self.model_network = QNetwork(self.obs_size,self.num_actions,
                                  kwargs.get('hidden_size',100),
                                  kwargs.get('hidden_layers',1),
                                  kwargs.get('learning_rate',.2))
    self.target_model_network = QNetwork(self.obs_size,self.num_actions,
                                         kwargs.get('hidden_size',100),
                                         kwargs.get('hidden_layers',1),
                                         kwargs.get('learning_rate',.2))
    self.target_model_network.model = QNetwork.copy_model(self.model_network.model)

    if policy == 'EpsilonGreedy':
      self.policy = Epsilon_greedy(self.model_network,len(actions),
                                   self.epsilon,self.min_epsilon,
                                   self.decay_rate,self.epoch_length)
      
  def learn(self,states,actions,next_states,rewards,done):

    if self.ExpRep:
        self.memory.observe(states,actions,rewards,done)
    
    else:
        self.states = states
        self.actions = actions
        self.next_states = next_states
        self.reward = rewards
        self.done   = done

  def update_model(self):

     if self.ExpRep:
           (states,actions,rewards,next_states,done) = self.memory.sample_minibatch(self.minibatch_size)

     else:
        
         states = self.states
         rewards = self.rewards
         next_states = self.next_states
         actions = self.actions
         done = self.done
     next_actions = []

     Q_prime = self.target_model_network.predict(next_states,self.minibatch_size)


     for row in range(Q_prime.shape[0]):

         best_next_actions  = np.argwhere(Q_prime[row] == np.amax(Q_prime[row]))

         next_actions.append(best_next_actions[np.random.choice(len(best_next_actions))].item())

     sx = np.arange(len(next_actions))

     Q = self.model_network.predict(states,self.minibatch_size)

     targets = rewards.reshape(Q[sx,actions].shape) + \
                self.gamma * Q[sx,next_actions] * \
                (1-done.reshape(Q[sx,actions].shape))
     Q[sx,actions] = targets

     loss = self.model_network.model.train_on_batch(states,Q)

     self.ddqn_update -=1

     if self.ddqn_update == 0:

        self.ddqn_update = self.ddqn_time

        self.target_model_network.model.set_weights(self.model_network.model.get_weights())

     return loss

  def act(self,state,policy):

        raise NotImplementedError







In [ ]:
class DefenderAgent(Agent):
  def __init__(self,actions,obs_size,policy="EpsilonGreedy",**kwargs):
     super().__init__(actions,obs_size,policy="EpsilonGreedy",**kwargs)

  def act(self,states):

     actions = self.policy.get_actions(states)
     return actions


class AttackAgent(Agent):

  def __init__(self,actions,obs_size,policy="EpsilonGreedy",**kwargs):
     super().__init__(actions,obs_size,policy="EpsilonGreedy",**kwargs)

  def act(self,states):

     actions = self.policy.get_actions(states)
     return actions





In [ ]:
class RLenv(data_cls):
  def __init__(self,train_test,**kwargs):
    data_cls.__init__(self,train_test,**kwargs)
    data_cls._load_df(self)
    self.data_shape = data_cls.get_shape(self)
    self.batch_size = kwargs.get('batch_size',1)
    self.iterations_episode = kwargs.get('iterations_episode',10)

    if self.batch_size =='full':
         self.batch_size = int(self.data_shape[0]/iterations_episode)

  def _update_state(self):        
        self.states,self.labels = data_cls.get_batch(self)
        
        # Update statistics
        self.true_labels += np.sum(self.labels).values

  def reset(self):
        # Statistics
    self.def_true_labels = np.zeros(len(self.attack_types),dtype=int)
    self.def_estimated_labels = np.zeros(len(self.attack_types),dtype=int)
    self.att_true_labels = np.zeros(len(self.attack_names),dtype=int)
        
    self.state_numb = 0
    print("hello in reset state")
    data_cls._load_df(self) # Reload and random index
    print("after load df")
    self.states,self.labels = data_cls.get_batch(self,self.batch_size)
    print("states") 
    self.total_reward = 0
    self.steps_in_episode = 0
    return self.states.values 



  def act(self,defender_actions,attack_actions):
     self.att_reward = np.zeros(len(attack_actions))
     self.def_reward = np.zeros(len(defender_actions))

     attack = [self.attack_types.index(self.attack_map[self.attack_names[att]]) for att in attack_actions]

     self.def_reward = (np.asarray(defender_actions)==np.asarray(attack))
     self.att_reward = (np.asarray(defender_actions)!=np.asarray(attack))


     self.def_estimated_labels += np.bincount(defender_actions,minlength=len(self.attack_types))

     for act in attack_actions:
        self.def_true_labels[self.attack_types.index(self.attack_map[self.attack_names[act]])] += 1

     attack_actions = attacker_agent.act(self.states)

     self.states = env.get_states(attack_actions)

     self.done = np.zeros(len(attack_actions),dtype=bool)


     return self.states,self.def_reward,self.att_reward,attack_actions,self.done

  def get_states(self,attacker_actions):
    first = True
    for attack in attacker_actions:
      if first:
        minibatch = (self.df[self.df[self.attack_names[attack]]== 1].sample(1))
        first = False
      else:
        minibatch = minibatch.append(self.df[self.df[self.attack_names[attack]]==1].sample(1))
    
    self.labels = minibatch[self.attack_names]
    minibatch.drop(self.all_attack_names,axis=1,inplace=True)
    self.states = minibatch

    return self.states


     

In [ ]:
if __name__ == "__main__":

  kdd_20_path = '/content/datasets/datasets/NSL/KDDTrain+_20Percent.txt'
  kdd_train = '/content/datasets/datasets/NSL/KDDTrain+.txt'
  kdd_test = '/content/datasets/datasets/NSL/KDDTest+.txt'

  formated_train_path = "/content/datasets/datasets/formated/formated_train_adv.data.txt"
  formated_test_path = "/content/datasets/datasets/formated/formated_test_adv.data.txt"
   
  batch_size = 1

  minibatch_size = 100

  ExpRep = True

  iterations_episode = 100


  env = RLenv('train',train_path=kdd_train,test_path=kdd_test,
              formated_train_path = formated_train_path,
              formated_test_path = formated_test_path,batch_size = batch_size,
              iterations_episode = iterations_episode)
  obs_size = env.data_shape[1]-len(env.all_attack_names)

  num_episodes = 100


  #Definition for the defensor agent

  defender_valid_actions = list(range(len(env.attack_types)))

  defender_num_actions = len(defender_valid_actions)


  def_epsilon = 1  #exploration
  min_epsilon = 0.01 #min value for exploration
  def_gamma = 0.001
  def_decay_rate = 0.99

  def_hidden_size = 100
  def_hidden_layers = 5

  def_learning_rate = .2

  defender_agent = DefenderAgent(defender_valid_actions,obs_size,"EpsilonGreedy",
                          epoch_length = iterations_episode,
                          epsilon = def_epsilon,
                          min_epsilon = min_epsilon,
                          decay_rate = def_decay_rate,
                          gamma = def_gamma,
                          hidden_size=def_hidden_size,
                          hidden_layers=def_hidden_layers,
                          minibatch_size = minibatch_size,
                          mem_size = 1000,
                          learning_rate=def_learning_rate,
                          ExpRep=ExpRep)
  
  #definitation of Attacker agent

  attack_valid_actions = list(range(len(env.attack_names)))
  attack_num_actions = len(attack_valid_actions)

  att_epsilon = 1
  min_epsilon = 0.8

  att_gamma = 0.001

  att_decay_rate = 0.99

  att_hidden_layers = 3
  att_hidden_size = 100
  att_learning_rate = 0.2

  
  attacker_agent = AttackAgent(attack_valid_actions,obs_size,"EpsilonGreedy",
                          epoch_length = iterations_episode,
                          epsilon = att_epsilon,
                          min_epsilon = min_epsilon,
                          decay_rate = att_decay_rate,
                          gamma = att_gamma,
                          hidden_size=att_hidden_size,
                          hidden_layers=att_hidden_layers,
                          minibatch_size = minibatch_size,
                          mem_size = 1000,
                          learning_rate=att_learning_rate,
                          ExpRep=ExpRep)
  
  att_reward_chain = []
  def_reward_chain = []
  att_loss_chain = []
  def_loss_chain = []
  def_total_reward_chain = []
  att_total_reward_chain = []

  print("-------------------------------------------------------------------------------")
  print("Total epoch: {} | Iterations in epoch: {}"
          "| Minibatch from mem size: {} | Total Samples: {}|".format(num_episodes,
                         iterations_episode,minibatch_size,
                         num_episodes*iterations_episode))
  print("-------------------------------------------------------------------------------")
  print("Dataset shape: {}".format(env.data_shape))
  print("-------------------------------------------------------------------------------")
  print("Attacker parameters: Num_actions={} | gamma={} |" 
          " epsilon={} | ANN hidden size={} | "
          "ANN hidden layers={}|".format(attack_num_actions,
                             att_gamma,att_epsilon, att_hidden_size,
                             att_hidden_layers))
  print("-------------------------------------------------------------------------------")
  print("Defense parameters: Num_actions={} | gamma={} | "
          "epsilon={} | ANN hidden size={} |"
          " ANN hidden layers={}|".format(defender_num_actions,
                              def_gamma,def_epsilon,def_hidden_size,
                              def_hidden_layers))
  print("-------------------------------------------------------------------------------")

    # Main loop
  for epoch in range(num_episodes):
      start_time = time.time()
      att_loss = 0.
      def_loss = 0.
      def_total_reward_by_episode = 0
      att_total_reward_by_episode = 0
        # Reset enviromet, actualize the data batch with random state/attacks
      states = env.reset()
        
        # Get actions for actual states following the policy
      attack_actions = attacker_agent.act(states)
      states = env.get_states(attack_actions)    

        
        
        
        
      done = False
       

        # Iteration in one episode
      for i_iteration in range(iterations_episode):
            
            
            # apply actions, get rewards and new state
          act_time = time.time()  
          defender_actions = defender_agent.act(states)
            #Enviroment actuation for this actions
          next_states,def_reward, att_reward,next_attack_actions, done = env.act(defender_actions,attack_actions)
            # If the epoch*batch_size*iterations_episode is largest than the df

            
          attacker_agent.learn(states,attack_actions,next_states,att_reward,done)
          defender_agent.learn(states,defender_actions,next_states,def_reward,done)
            
          act_end_time = time.time()
            
            # Train network, update loss after at least minibatch_learns
          if ExpRep and epoch*iterations_episode + i_iteration >= minibatch_size:
              def_loss += defender_agent.update_model()
              att_loss += attacker_agent.update_model()
          elif not ExpRep:
              def_loss += defender_agent.update_model()
              att_loss += attacker_agent.update_model()
                

          update_end_time = time.time()

            # Update the state
          states = next_states
          attack_actions = next_attack_actions
            
            
            # Update statistics
          def_total_reward_by_episode += np.sum(def_reward,dtype=np.int32)
          att_total_reward_by_episode += np.sum(att_reward,dtype=np.int32)
        

        # Update user view
      def_reward_chain.append(def_total_reward_by_episode) 
      att_reward_chain.append(att_total_reward_by_episode) 
      def_loss_chain.append(def_loss)
      att_loss_chain.append(att_loss) 

        
      end_time = time.time()
      print("\r\n|Epoch {:03d}/{:03d}| time: {:2.2f}|\r\n"
                "|Def Loss {:4.4f} | Def Reward in ep {:03d}|\r\n"
                "|Att Loss {:4.4f} | Att Reward in ep {:03d}|"
                .format(epoch, num_episodes,(end_time-start_time), 
                def_loss, def_total_reward_by_episode,
                att_loss, att_total_reward_by_episode))
        
        
      print("|Def Estimated: {}| Att Labels: {}".format(env.def_estimated_labels,
              env.def_true_labels))
        
    # Save trained model weights and architecture, used in test
  defender_agent.model_network.model.save_weights("/content/datasets/datasets/results/defender_agent_model.h5", overwrite=True)
  with open("/content/datasets/datasets/results/defender_agent_model.json", "w") as outfile:
    json.dump(defender_agent.model_network.model.to_json(), outfile)
        
        
    # Plot training results
  plt.figure(1)
  plt.subplot(211)
  plt.plot(np.arange(len(def_reward_chain)),def_reward_chain,label='Defense')
  plt.plot(np.arange(len(att_reward_chain)),att_reward_chain,label='Attack')
  plt.title('Total reward by episode')
  plt.xlabel('n Episode')
  plt.ylabel('Total reward')
  plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    
  plt.subplot(212)
  plt.plot(np.arange(len(def_loss_chain)),def_loss_chain,label='Defense')
  plt.plot(np.arange(len(att_loss_chain)),att_loss_chain,label='Attack')
  plt.title('Loss by episode')
  plt.xlabel('n Episode')
  plt.ylabel('loss')
  plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
  plt.tight_layout()
    #plt.show()
  plt.savefig('/content/datasets/datasets/results/train_adv.eps', format='eps', dpi=1000)






In [ ]:
import json
import numpy as np
import pandas as pd
from keras.models import model_from_json
#from adversarialAD import RLenv
import matplotlib.pyplot as plt
#from adversarialAD import huber_loss

import itertools
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import  confusion_matrix





def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')









batch_size = 10
formated_test_path = "/content/datasets/datasets/formated/formated_test_adv.data.txt"

with open("/content/datasets/datasets/results/defender_agent_model.json", "r") as jfile:
    model = model_from_json(json.load(jfile))
model.load_weights("/content/datasets/datasets/results/defender_agent_model.h5")
model.compile(loss=huber_loss,optimizer='sgd')

# Define environment, game, make sure the batch_size is the same in train
env = RLenv('test',formated_test_path = formated_test_path)


total_reward = 0    


true_labels = np.zeros(len(env.attack_types),dtype=int)
estimated_labels = np.zeros(len(env.attack_types),dtype=int)
estimated_correct_labels = np.zeros(len(env.attack_types),dtype=int)

#states , labels = env.get_sequential_batch(test_path,batch_size = env.batch_size)
states , labels = env.get_full()
q = model.predict(states)
actions = np.argmax(q,axis=1)        

maped=[]
for indx,label in labels.iterrows():
    maped.append(env.attack_types.index(env.attack_map[label.idxmax()]))

labels,counts = np.unique(maped,return_counts=True)
true_labels[labels] += counts



for indx,a in enumerate(actions):
    estimated_labels[a] +=1              
    if a == maped[indx]:
        total_reward += 1
        estimated_correct_labels[a] += 1


action_dummies = pd.get_dummies(actions)
posible_actions = np.arange(len(env.attack_types))
for non_existing_action in posible_actions:
    if non_existing_action not in action_dummies.columns:
        action_dummies[non_existing_action] = np.uint8(0)
labels_dummies = pd.get_dummies(maped)

normal_f1_score = f1_score(labels_dummies[0].values,action_dummies[0].values)
dos_f1_score = f1_score(labels_dummies[1].values,action_dummies[1].values)
probe_f1_score = f1_score(labels_dummies[2].values,action_dummies[2].values)
r2l_f1_score = f1_score(labels_dummies[3].values,action_dummies[3].values)
u2r_f1_score = f1_score(labels_dummies[4].values,action_dummies[4].values)
    

Accuracy = [normal_f1_score,dos_f1_score,probe_f1_score,r2l_f1_score,u2r_f1_score]
Mismatch = estimated_labels - true_labels

acc = float(100*total_reward/len(states))
print('\r\nTotal reward: {} | Number of samples: {} | Accuracy = {:.2f}%'.format(total_reward,
      len(states),acc))
outputs_df = pd.DataFrame(index = env.attack_types,columns = ["Estimated","Correct","Total","F1_score"])
for indx,att in enumerate(env.attack_types):
   outputs_df.iloc[indx].Estimated = estimated_labels[indx]
   outputs_df.iloc[indx].Correct = estimated_correct_labels[indx]
   outputs_df.iloc[indx].Total = true_labels[indx]
   outputs_df.iloc[indx].F1_score = Accuracy[indx]*100
   outputs_df.iloc[indx].Mismatch = abs(Mismatch[indx])

##############





        

    #%%

print(outputs_df)

fig, ax = plt.subplots()
width = 0.35
pos = np.arange(len(true_labels))
p1 = plt.bar(pos, estimated_correct_labels,width,color='g')
p1 = plt.bar(pos+width,
             (np.abs(estimated_correct_labels-true_labels)),width,
             color='r')
p2 = plt.bar(pos+width,np.abs(estimated_labels-estimated_correct_labels),width,
             bottom=(np.abs(estimated_correct_labels-true_labels)),
             color='b')


ax.set_xticks(pos+width/2)
ax.set_xticklabels(env.attack_types,rotation='vertical')
#ax.set_yscale('log')

#ax.set_ylim([0, 100])
ax.set_title('Test set scores, Acc = {:.2f}'.format(acc))
plt.legend(('Correct estimated','False negative','False positive'))
plt.tight_layout()
#plt.show()
plt.savefig('/content/datasets/datasets/test_adv_imp.svg', format='svg', dpi=1000)


#%% Agregated precision

aggregated_data_test = np.array(maped)

print('Performance measures on Test data')
print('Accuracy =  {:.4f}'.format(accuracy_score( aggregated_data_test,actions)))
print('F1 =  {:.4f}'.format(f1_score(aggregated_data_test,actions, average='weighted')))
print('Precision_score =  {:.4f}'.format(precision_score(aggregated_data_test,actions, average='weighted')))
print('recall_score =  {:.4f}'.format(recall_score(aggregated_data_test,actions, average='weighted')))

cnf_matrix = confusion_matrix(aggregated_data_test,actions)
np.set_printoptions(precision=2)
plt.figure()
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=env.attack_types, normalize=True,
                      title='Normalized confusion matrix')
plt.savefig('/content/datasets/datasets/confusion_matrix_adversarial.svg', format='svg', dpi=1000)

In [ ]:
df2 = pd.read_csv('/content/datasets/datasets/formated/formated_train_adv.data.txt')
df2.head(10)

,duration,src_bytes,dst_bytes,land_f,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,icmp,tcp,...,apache2,back,buffer_overflow,ftp_write,guess_passwd,httptunnel,imap,ipsweep,land,loadmodule,mailbomb,mscan,multihop,named,neptune,nmap,normal,perl,phf,pod,portsweep,processtable,ps,rootkit,saint,satan,sendmail,smurf,snmpgetattack,snmpguess,spy,sqlattack,teardrop,udpstorm,warezclient,warezmaster,worm,xlock,xsnoop,xterm
0,0.000000,2.115997e-07,1.007834e-05,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.062745,1.000000,1.00,0.00,0.06,0.07,0.0,0.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.103718,0.033268,1.0,1.0,0.0,0.0,0.32,0.08,0.00,1.000000,0.258824,0.26,0.02,0.01,0.00,1.0,1.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.000000,2.115997e-07,1.885586e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.035225,0.037182,0.0,0.0,0.0,0.0,1.00,0.00,0.11,0.207843,1.000000,1.00,0.00,0.02,0.03,0.0,0.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.000000,1.456560e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.090196,0.278431,0.91,0.09,0.91,0.03,0.0,0.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.000035,3.374001e-06,2.534472e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.411765,0.670588,0.87,0.04,0.01,0.01,0.0,0.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.000000,3.952277e-05,6.346868e-06,0.0,0.0,0.0,0.019802,0.0,1.0,0.000134,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.005871,0.005871,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.592157,0.592157,1.00,0.00,0.01,0.00,0.0,0.0,0.01,0.01,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0.000000,2.536298e-08,7.252255e-08,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.005871,0.005871,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.592157,0.819608,0.90,0.01,0.90,0.01,0.0,0.0,0.07,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.557730,0.039139,1.0,1.0,0.0,0.0,0.07,0.06,0.00,1.000000,0.137255,0.14,0.06,0.00,0.00,1.0,1.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0.000000,3.952277e-05,6.346868e-06,0.0,0.0,0.0,0.019802,0.0,1.0,0.000134,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.007828,0.007828,0.0,0.0,0.0,0.0,1.00,0.00,0.00,0.184314,0.184314,1.00,0.00,0.02,0.00,0.0,0.0,0.04,0.04,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.532290,0.029354,1.0,1.0,0.0,0.0,0.06,0.07,0.00,1.000000,0.058824,0.06,0.08,0.00,0.00,1.0,1.0,0.00,0.00,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
